#### 2 CPI data (Si Qin)

Consumer CPI data is obtained from U.S. Bureau of Labory Statistics (BLS) public FTP server https://download.bls.gov/pub/time.series/cu/
The raw downloaded data include: 
1. cu.data.0.Current # Contains all current CPI data series
2. cu.series         # Metadata for each series

In [1]:
# Load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Load data
cpi_data0 = pd.read_csv("CPI_data/cu.data.0.Current", sep='\t', engine='python')
cpi_meta = pd.read_csv("CPI_data/cu.series", sep='\t', engine='python')
cpi_data0.columns = cpi_data0.columns.str.strip()
cpi_meta.columns = cpi_meta.columns.str.strip()

In [3]:
print(cpi_data0.head())
print(cpi_data0.columns.tolist())
print(cpi_data0.shape)
print(len(cpi_data0['series_id'].unique()))
# print out all the years and periods
for col in ['year','period']:
	print(f"Unique values in {col}:")
	print(len(cpi_data0[col].unique()))
	print('=' * 40)

cpi_data0['period'].unique() # M13 is the annual average

           series_id  year period  value  footnote_codes
0  CUSR0000SA0        1997    M01  159.4             NaN
1  CUSR0000SA0        1997    M02  159.7             NaN
2  CUSR0000SA0        1997    M03  159.8             NaN
3  CUSR0000SA0        1997    M04  159.9             NaN
4  CUSR0000SA0        1997    M05  159.9             NaN
['series_id', 'year', 'period', 'value', 'footnote_codes']
(1107696, 5)
7935
Unique values in year:
29
Unique values in period:
16


array(['M01', 'M02', 'M03', 'M04', 'M05', 'M06', 'M07', 'M08', 'M09',
       'M10', 'M11', 'M12', 'M13', 'S01', 'S02', 'S03'], dtype=object)

In [4]:
print(cpi_meta.head())
cpi_meta.head()
#cpi_meta.shape
print(len(cpi_meta['series_id'].unique()))

           series_id area_code item_code seasonal periodicity_code base_code  \
0  CUSR0000SA0            0000       SA0        S                R         S   
1  CUSR0000SA0E           0000      SA0E        S                R         S   
2  CUSR0000SA0L1          0000     SA0L1        S                R         S   
3  CUSR0000SA0L12         0000    SA0L12        S                R         S   
4  CUSR0000SA0L12E        0000   SA0L12E        S                R         S   

   base_period                                       series_title  \
0  1982-84=100  All items in U.S. city average, all urban cons...   
1  1982-84=100  Energy in U.S. city average, all urban consume...   
2  1982-84=100  All items less food in U.S. city average, all ...   
3  1982-84=100  All items less food and shelter in U.S. city a...   
4  1982-84=100  All items less food, shelter, and energy in U....   

   footnote_codes  begin_year begin_period  end_year end_period  
0             NaN        1947         

In [5]:
# Extract the series id of U.S. city average, all urban consumers, and not seasonally adjusted
# also exclude all the services
service_keywords = ['services', 'transportation', 'medical', 'education', 'care', 'repair', 'rent', 'tuition', 'fare', 'postage', 'service', 'insurance', 'fee' , 'admission']
pattern = '|'.join(service_keywords)

unadjusted_id = cpi_meta['series_id'][
    cpi_meta['series_title'].str.contains('U.S. city average', case=False) &
    cpi_meta['series_title'].str.contains('all urban consumers', case=False) &
    (cpi_meta['seasonal'] == 'U') &        # Not seasonally adjusted    
    ~cpi_meta['series_title'].str.contains(pattern, case=False, na=False)
    ]

print(len(unadjusted_id))
# Aggregate the CPI data to annual price and use unadjusted value
unj_cpi_data = cpi_data0[cpi_data0['series_id'].isin(unadjusted_id) & (cpi_data0['period'] == 'M13')].drop(columns=['footnote_codes','period'])
print(unj_cpi_data.shape)
print(len(unj_cpi_data['series_id'].unique()))

573
(7729, 3)
290


In [6]:
# Targe label: Label big price increase (>5%) from 2017 to 2021 as 1, otherwise 0
# make wide table
wide_cpi = unj_cpi_data.pivot(index='series_id', columns='year', values='value').reset_index()
# add labels
wide_cpi['label'] = np.where((wide_cpi[2021]/wide_cpi[2017]-1) > 0.05, 1, 0)
print(wide_cpi['label'].value_counts())
# merge the description from meta data to the aggreaged data
cpi_all = pd.merge(wide_cpi, cpi_meta[['series_id','series_title']], on = 'series_id', how='left')

label
1    184
0    106
Name: count, dtype: int64


In [7]:
for title in cpi_all[cpi_all['label'] == 1]['series_title']:
    print(title)

All items - old base in U.S. city average, all urban consumers, not seasonally adjusted
All items in U.S. city average, all urban consumers, not seasonally adjusted
Energy in U.S. city average, all urban consumers, not seasonally adjusted
All items less food in U.S. city average, all urban consumers, not seasonally adjusted
All items less food and shelter in U.S. city average, all urban consumers, not seasonally adjusted
All items less food, shelter, and energy in U.S. city average, all urban consumers, not seasonally adjusted
All items less food, shelter, energy, and used cars and trucks in U.S. city average, all urban consumers, not seasonally adjusted
All items less food and energy in U.S. city average, all urban consumers, not seasonally adjusted
All items less shelter in U.S. city average, all urban consumers, not seasonally adjusted
All items less energy in U.S. city average, all urban consumers, not seasonally adjusted
Commodities in U.S. city average, all urban consumers, not s

Most of the items with more than 5% increase form 2017 to 2021 are food, housing. Relevent items to the goods imported from China are:  
- Men's underwear, nightwear, swimwear, and accessories. 
- Men's pants and shorts. 
- Floor coverings. 
- Furniture and bedding. 
- Bedroom furniture. 
- Living room, kitchen, and dining room furniture. 
- Other furniture. 
- Appliances. 
- Major appliances. 
- Other appliances. 
- Tools, hardware, outdoor equipment and supplies. 
- Tools, hardware and supplies. 
- Outdoor equipment and supplies. 
- Housekeeping supplies. 
- Household cleaning products. 
- Household paper products. 
- Household operations. 
- Sports vehicles including bicycles
- New and used motor vehicles 
- New vehicles 
- Used cars and trucks


In [ ]:
# Define keyword groups for CPI categories
cpi_keywords = {
    "men_clothing": ["underwear", "nightwear", "swimwear", "pants", "shorts"],
    "floor_coverings": ["carpet", "rug", "floor covering"],
    "furniture": ["furniture", "bedding"],
    "appliances": ["appliance", "refrigerator", "washer", "dryer", "microwave"],
    "tools_hardware": ["tools", "hardware", "lawn", "garden", "outdoor equipment"],
    "cleaning_supplies": ["cleaning", "detergent", "housekeeping"],
    "paper_products": ["toilet paper", "paper towel", "tissue"],
    "household_operations": ["household operation"],
    "sports_vehicles": ["bicycle", "sports vehicle"],
    "motor_vehicles": ["automobile", "car", "passenger vehicle", "truck", "new car", "used car"]
}

# Convert description to lowercase for case-insensitive matching
tariff_df["Description_lower"] = tariff_df["Description"].str.lower()

# Initialize label column
tariff_df["label"] = 0

# Match descriptions to keyword lists
for category, keywords in cpi_keywords.items():
    for keyword in keywords:
        match_mask = tariff_df["Description_lower"].str.contains(keyword, na=False)
        tariff_df.loc[match_mask, "label"] = 1
        tariff_df.loc[match_mask, "matched_category"] = category

# Optional: keep only HTS and match info
matched_df = tariff_df[tariff_df["label"] == 1][["HTS", "Description", "matched_category"]].drop_duplicates()

# Save result
matched_df.to_csv("matched_hts_to_cpi.csv", index=False)

print("Matching complete. Saved to 'matched_hts_to_cpi.csv'.")

NameError: name 'tariff_df' is not defined